# Historical Fraud Report

### Submit a (api) request for a date interval in the last 30 days and get a summary report.

In [8]:
from os import path, makedirs
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import pyarrow as pa

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [10]:
target_path = path.join('/v3io', 'bigdata', 'parquet', 'paysim')

In [18]:
paysim = pq.read_table(path.join(target_path, 'paysim')).to_pandas()

In [20]:
paysim.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [21]:
paysim.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 
                       'newbalanceOrig':'newBalanceOrig',
                       'oldbalanceDest':'oldBalanceDest',
                       'newbalanceDest':'newBalanceDest'},
              inplace=True)

In [22]:
format_mapping={
    'amount': '${:,.2f}',
    'oldBalanceOrig': '${:,.2f}',
    'newBalanceOrig': '${:,.2f}',
    'oldBalanceDest': '${:,.2f}',
    'newBalanceDest': '${:,.2f}',}

In [23]:
for key, value in format_mapping.items():
    paysim[key] = paysim[key].apply(value.format)

In [24]:
paysim.head()

,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,"$9,839.64",C1231006815,"$170,136.00","$160,296.36",M1979787155,$0.00,$0.00,0,0
1,1,PAYMENT,"$1,864.28",C1666544295,"$21,249.00","$19,384.72",M2044282225,$0.00,$0.00,0,0
2,1,TRANSFER,$181.00,C1305486145,$181.00,$0.00,C553264065,$0.00,$0.00,1,0
3,1,CASH_OUT,$181.00,C840083671,$181.00,$0.00,C38997010,"$21,182.00",$0.00,1,0
4,1,PAYMENT,"$11,668.14",C2048537720,"$41,554.00","$29,885.86",M1230701703,$0.00,$0.00,0,0


[Apparently](predicting-fraud-in-financial-payment-services.ipynb#fraud-trans),
>fraud is committed by first transferring out funds
to another account which subsequently cashes it out

In [25]:
print('\n The types of fraudulent transactions are {}'.format(\
list(paysim.loc[paysim.isFraud == 1].type.drop_duplicates().values))) # only 'CASH_OUT' & 'TRANSFER'

dfFraudTransfer = paysim.loc[(paysim.isFraud == 1) & (paysim.type == 'TRANSFER')]
dfFraudCashout = paysim.loc[(paysim.isFraud == 1) & (paysim.type == 'CASH_OUT')]

print ('\n The number of fraudulent TRANSFERs = {}'.\
       format(len(dfFraudTransfer))) # 4097

print ('\n The number of fraudulent CASH_OUTs = {}'.\
       format(len(dfFraudCashout))) # 4116


 The types of fraudulent transactions are ['TRANSFER', 'CASH_OUT']

 The number of fraudulent TRANSFERs = 4097

 The number of fraudulent CASH_OUTs = 4116


In [29]:
print('\nThe type of transactions in which isFlaggedFraud is set: \
{}'.format(list(paysim.loc[paysim.isFlaggedFraud == 1].type.drop_duplicates()))) # only 'TRANSFER'

dfTransfer = paysim.loc[paysim.type == 'TRANSFER']
dfFlagged = paysim.loc[paysim.isFlaggedFraud == 1]
dfNotFlagged = paysim.loc[paysim.isFlaggedFraud == 0]

print(f'\nMin amount transacted when isFlaggedFraud is set= {dfFlagged.amount.min()}') # 353874.22

print(f'\nMax amount transacted in a TRANSFER where isFlaggedFraud is not set=\
{dfTransfer.loc[dfTransfer.isFlaggedFraud == 0].amount.max()}') # 92445516.64


The type of transactions in which isFlaggedFraud is set: ['TRANSFER']

Min amount transacted when isFlaggedFraud is set= $1,343,002.08

Max amount transacted in a TRANSFER where isFlaggedFraud is not set=$999,988.34


In [30]:
dfTransfer.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
               'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'},
              inplace=True)

/User/.pythonlibs/lib/python3.6/site-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [31]:
print('\nThe number of TRANSFERs where isFlaggedFraud = 0, yet oldBalanceDest = 0 and\
 newBalanceDest = 0: {}'.\
format(len(dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & \
(dfTransfer.oldBalanceDest == 0) & (dfTransfer.newBalanceDest == 0)]))) # 4158


The number of TRANSFERs where isFlaggedFraud = 0, yet oldBalanceDest = 0 and newBalanceDest = 0: 0


In [34]:
print('\nMin, Max of oldBalanceOrig for isFlaggedFraud = 1 TRANSFERs: {}'.\
format([dfFlagged.oldBalanceOrig.min(), dfFlagged.oldBalanceOrig.max()]))

print('\nMin, Max of oldBalanceOrig for isFlaggedFraud = 0 TRANSFERs where \
oldBalanceOrig = \
newBalanceOrig: {}'.format(\
[dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & (dfTransfer.oldBalanceOrig \
== dfTransfer.newBalanceOrig)].oldBalanceOrig.min(), \
dfTransfer.loc[(dfTransfer.isFlaggedFraud == 0) & (dfTransfer.oldBalanceOrig \
               == dfTransfer.newBalanceOrig)].oldBalanceOrig.max()]))


Min, Max of oldBalanceOrig for isFlaggedFraud = 1 TRANSFERs: ['$1,343,002.08', '$536,624.41']

Min, Max of oldBalanceOrig for isFlaggedFraud = 0 TRANSFERs where oldBalanceOrig = newBalanceOrig: ['$0.00', '$575,667.54']


In [35]:
print('\nHave originators of transactions flagged as fraud transacted more than \
once? {}'\
.format((dfFlagged.nameOrig.isin(pd.concat([dfNotFlagged.nameOrig, \
                                    dfNotFlagged.nameDest]))).any())) # False

print('\nHave destinations for transactions flagged as fraud initiated\
 other transactions? \
{}'.format((dfFlagged.nameDest.isin(dfNotFlagged.nameOrig)).any())) # False

# Since only 2 destination accounts of 16 that have 'isFlaggedFraud' set have been
# destination accounts more than once,
# clearly 'isFlaggedFraud' being set is independent of whether a 
# destination account has been used before or not

print('\nHow many destination accounts of transactions flagged as fraud have been \
destination accounts more than once?: {}'\
.format(sum(dfFlagged.nameDest.isin(dfNotFlagged.nameDest)))) # 2


Have originators of transactions flagged as fraud transacted more than once? False

Have destinations for transactions flagged as fraud initiated other transactions? False

How many destination accounts of transactions flagged as fraud have been destination accounts more than once?: 2


In [36]:
print('\nAre there any merchants among originator accounts for CASH_IN \
transactions? {}'.format(\
(paysim.loc[paysim.type == 'CASH_IN'].nameOrig.str.contains('M')).any())) # False


Are there any merchants among originator accounts for CASH_IN transactions? False


In [37]:
print('\nAre there any merchants among destination accounts for CASH_OUT \
transactions? {}'.format(\
(paysim.loc[paysim.type == 'CASH_OUT'].nameDest.str.contains('M')).any())) # False


Are there any merchants among destination accounts for CASH_OUT transactions? False


In [38]:
print('\nAre there merchants among any originator accounts? {}'.format(\
      paysim.nameOrig.str.contains('M').any())) # False

print('\nAre there any transactions having merchants among destination accounts\
 other than the PAYMENT type? {}'.format(\
(paysim.loc[paysim.nameDest.str.contains('M')].type != 'PAYMENT').any())) # False


Are there merchants among any originator accounts? False

Are there any transactions having merchants among destination accounts other than the PAYMENT type? False


In [39]:
print('\nWithin fraudulent transactions, are there destinations for TRANSFERS \
that are also originators for CASH_OUTs? {}'.format(\
(dfFraudTransfer.nameDest.isin(dfFraudCashout.nameOrig)).any())) # False
dfNotFraud = paysim.loc[paysim.isFraud == 0]


Within fraudulent transactions, are there destinations for TRANSFERS that are also originators for CASH_OUTs? False


In [40]:
print('\nFraudulent TRANSFERs whose destination accounts are originators of \
genuine CASH_OUTs: \n\n{}'.format(dfFraudTransfer.loc[dfFraudTransfer.nameDest.\
isin(dfNotFraud.loc[dfNotFraud.type == 'CASH_OUT'].nameOrig.drop_duplicates())]))


Fraudulent TRANSFERs whose destination accounts are originators of genuine CASH_OUTs: 

         step      type         amount     nameOrig oldBalanceOrig  \
1030443    65  TRANSFER  $1,282,971.57  C1175896731  $1,282,971.57   
6039814   486  TRANSFER    $214,793.32  C2140495649    $214,793.32   
6362556   738  TRANSFER    $814,689.88  C2029041842    $814,689.88   

        newBalanceOrig     nameDest oldBalanceDest newBalanceDest  isFraud  \
1030443          $0.00  C1714931087          $0.00          $0.00        1   
6039814          $0.00   C423543548          $0.00          $0.00        1   
6362556          $0.00  C1023330867          $0.00          $0.00        1   

         isFlaggedFraud  
1030443               0  
6039814               0  
6362556               0  


In [41]:
print('\nFraudulent TRANSFER to C423543548 occured at step = 486 whereas \
genuine CASH_OUT from this account occured earlier at step = {}'.format(\
dfNotFraud.loc[(dfNotFraud.type == 'CASH_OUT') & (dfNotFraud.nameOrig == \
                           'C423543548')].step.values)) # 185


Fraudulent TRANSFER to C423543548 occured at step = 486 whereas genuine CASH_OUT from this account occured earlier at step = [185]
